In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
from nltk.stem import WordNetLemmatizer
from sklearn import metrics

In [44]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [55]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [25]:
combined = pd.read_csv("cp2077_reviews_cleaned.csv.zip", compression="zip")

# drop column Date Timestamp Created
combined = combined.drop(columns=["Date Timestamp Created"])

# make sure reviews are strings
combined["Review"] = combined["Review"].astype("str")

#split combined into train and test
train = combined.sample(frac=0.8, random_state=200).reset_index(drop=True)
test = combined.drop(train.index).reset_index(drop=True)

In [6]:
train_X_non = train['Review']   # '0' refers to the review text
train_y = train['Recommended or Not Recommended']   # '1' corresponds to Label (1 - positive and 0 - negative)
test_X_non = test['Review']
test_y = test['Recommended or Not Recommended']
train_X=[]
test_X=[]

In [9]:
#text pre processing for train data
for i in range(0, len(train_X_non)):
    review = re.sub('[^a-zA-Z]', ' ', train_X_non[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)

In [10]:
#text pre processing for test data
for i in range(0, len(test_X_non)):
    review = re.sub('[^a-zA-Z]', ' ', test_X_non[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)

In [38]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)
#applying tf idf to training data
X_train_tf = tf_idf.transform(train_X)

In [39]:
#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(test_X)

In [40]:
#naive bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)

MultinomialNB()

In [41]:
#predicted y
y_pred = naive_bayes_classifier.predict(X_test_tf)

In [45]:
result_nb = metrics.classification_report(test_y, y_pred, target_names=['True', 'False'])

# print the accuracy score
print("Accuracy:",metrics.accuracy_score(test_y, y_pred))

Accuracy: 0.9419047619047619


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SVM

In [46]:
combined.columns

Index(['Review', 'Recommended or Not Recommended'], dtype='object')

In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(combined["Review"], combined["Recommended or Not Recommended"], test_size=0.25, random_state=30)
print("Train: ",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (7875,) (7875,) Test:  ((2625,), (2625,))


In [48]:
print("TFIDF Vectorizer……")
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

TFIDF Vectorizer……


In [29]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

In [30]:
clf.fit(tf_x_train,Y_train)

LinearSVC(random_state=0)

In [31]:
y_test_pred=clf.predict(tf_x_test)


In [49]:
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)
# print(report)

# print accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: ",accuracy_score(Y_test, y_test_pred))


Accuracy:  0.8365714285714285


# Logistic Regression

In [51]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,solver="saga")

In [52]:
clf.fit(tf_x_train,Y_train)

LogisticRegression(max_iter=1000, solver='saga')

In [53]:
y_test_pred=clf.predict(tf_x_test)

In [54]:
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)

# print accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: ",accuracy_score(Y_test, y_test_pred))

Accuracy:  0.8342857142857143


In [11]:
import numpy as np
import pandas as pd

In [3]:
like_tfidf = np.log((10000 + 1)) * np.log(4000/1500)
the_tfidf = np.log((50000 + 1)) * np.log(4000/3000)

print(like_tfidf, the_tfidf)



9.033869345050624 3.1126619939118374


In [9]:
# this = 100 (+) 150 (-) 1000  || 25
the_tfidf = np.log((100 + 1)) * np.log(1000/25)
print(the_tfidf)

the_tfidf1 = np.log((150 + 1)) * np.log(1000/25)
print(the_tfidf1)

# adore = 60 (+) 3 (-) 1000  || 63
the_tfidf2 = np.log((60 + 1)) * np.log(1000/63)
print(the_tfidf2)

the_tfidf3 = np.log((3 + 1)) * np.log(1000/63)
print(the_tfidf3)

17.024623252835415
18.508140505566697
11.365006374001092
3.8325778826925108


In [12]:
df_recom = pd.read_csv("bow_recom.csv")
df_not_recom = pd.read_csv("bow_not_recom.csv")

print(df_recom)
print(df_not_recom)

           Word  Count
0          game  12424
1           the   3379
2          like   2990
3         story   2925
4          bugs   2874
...         ...    ...
27310  whileall      1
27311  slowdown      1
27312    plying      1
27313   ounlike      1
27314   wicther      1

[27315 rows x 2 columns]
              Word  Count
0             game  16413
1             like   4647
2              the   4520
3             even   3207
4            story   2883
...            ...    ...
33700      fpslots      1
33701      itediti      1
33702      shortit      1
33703  endingstill      1
33704        trial      1

[33705 rows x 2 columns]


In [17]:
witcher = df_recom.loc[df_recom["Word"]=="witcher",:]
witcher1 = df_not_recom.loc[df_not_recom["Word"]=="witcher",:]
print(witcher)
print(witcher1)

       Word  Count
86  witcher    542
       Word  Count
95  witcher    691


In [28]:
the = df_recom.loc[df_recom["Word"]=="the",:]
the1 = df_not_recom.loc[df_not_recom["Word"]=="the",:]
print(the)
print(the1)

  Word  Count
1  the   3379
  Word  Count
2  the   4520


In [33]:
love = df_recom.loc[df_recom["Word"]=="love",:]
love1 = df_not_recom.loc[df_not_recom["Word"]=="love",:]
print(love)
print(love1)

    Word  Count
58  love    732
     Word  Count
236  love    346


In [34]:
a = love['Count'].item()
b = love1['Count'].item()
sum1 = df_recom['Count'].sum()
sum2 = df_not_recom['Count'].sum()

# adore = 60 (+) 3 (-) 1000  || 63
the_tfidf2_pos = np.log((a + 1)) * np.log((sum1+sum2)/(a+b))
print(the_tfidf2_pos)

the_tfidf3_neg = np.log((b + 1)) * np.log((sum1+sum2)/(a+b))
print(the_tfidf3_neg)

43.50921884200013
38.57722163933114


In [29]:
a = the['Count'].item()
b = the1['Count'].item()
sum1 = df_recom['Count'].sum()
sum2 = df_not_recom['Count'].sum()

# adore = 60 (+) 3 (-) 1000  || 63
the_tfidf2_pos = np.log((a + 1)) * np.log((sum1+sum2)/(a+b))
print(the_tfidf2_pos)

the_tfidf3_neg = np.log((b + 1)) * np.log((sum1+sum2)/(a+b))
print(the_tfidf3_neg)

37.4065810888035
38.74555213366559


In [27]:
a = witcher['Count'].item()
b = witcher1['Count'].item()
sum1 = df_recom['Count'].sum()
sum2 = df_not_recom['Count'].sum()

# adore = 60 (+) 3 (-) 1000  || 63
the_tfidf2_pos = np.log((a + 1)) * np.log((sum1+sum2)/(a+b))
print(the_tfidf2_pos)

the_tfidf3_neg = np.log((b + 1)) * np.log((sum1+sum2)/(a+b))
print(the_tfidf3_neg)

40.6844604767177
42.25105724355068
